In [1]:
import pyspark
from operator import add, sub
from time import sleep
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.streaming.kafka import KafkaUtils
import json
from pyspark import SparkContext, SparkConf


In [2]:
# conf = SparkConf().set("spark.jars", "/path-to-jar/spark-streaming-kafka-0-8-assembly_2.11-2.2.1.jar")

# sc = SparkContext( conf=conf)
# spark = SparkSession.builder.appName('my_awes').master("spark://bphani.local:7077").config("spark.jars", "/Users/phaneendra/Downloads/spark-sql-kafka-0-10_2.11-2.4.4.jar,/Users/phaneendra/Downloads/kafka-clients-2.4.0.jar").getOrCreate()
spark = SparkSession.builder.appName('my_awes').master("local").config("spark.jars", "/Users/phaneendra/Downloads/spark-sql-kafka-0-10_2.11-2.4.4.jar,/Users/phaneendra/Downloads/kafka-clients-2.4.0.jar").getOrCreate()



In [5]:
df = spark.read.csv('/Users/phaneendra/Downloads/100000SalesRecords.csv', inferSchema=True, header=True)

In [11]:
df.columns

['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'Order ID',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

In [10]:

df.selectExpr("write_kafka_test", "CAST(Country AS STRING)", "CAST(Region AS STRING)") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .save()



AnalysisException: "cannot resolve '`write_kafka_test`' given input columns: [Total Profit, Item Type, Units Sold, Total Cost, Ship Date, Order Priority, Order Date, Unit Price, Unit Cost, Country, Region, Sales Channel, Order ID, Total Revenue]; line 1 pos 0;\n'Project ['write_kafka_test, cast(Country#49 as string) AS Country#88, cast(Region#48 as string) AS Region#89]\n+- AnalysisBarrier\n      +- Relation[Region#48,Country#49,Item Type#50,Sales Channel#51,Order Priority#52,Order Date#53,Order ID#54,Ship Date#55,Units Sold#56,Unit Price#57,Unit Cost#58,Total Revenue#59,Total Cost#60,Total Profit#61] csv\n"

In [3]:
dft1 = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("subscribe", "topic-one").option("startingOffsets", "latest").load()



In [4]:
dft1.writeStream.format("kafka").option("topic", "topic-one").option("kafka.bootstrap.servers", "localhost:9092").option("checkpointLocation","/tmp/").start()




In [3]:
prob_schema = StructType().add("CUST_ID", "string").add("OUTBUFFER__1", "string")
# Read CSV files from set path
dfCSV = spark.readStream.option("sep", ",").option("header", "true").schema(prob_schema).csv("/Users/phaneendra/Downloads/prob*.csv")


In [4]:
dfCSV.createOrReplaceTempView("prob")
total = spark.sql("select count(CUST_ID) from prob")


In [5]:
dfCSV \
.write  \
.format("kafka") \
.option("kafka.batch.size", 5000) \
.option("kafka.bootstrap.servers", "localhost:9092") \
.option("kafka.request.timeout.ms", 120000) \
.option("topic", "raw") \
.option("checkpointLocation", "/mnt/telemetry/cp.txt") \
.save()

AnalysisException: "'write' can not be called on streaming Dataset/DataFrame;"

In [ ]:
query = total.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()

In [ ]:
query.close()

In [9]:
dfCSV.selectExpr("CAST(CUST_ID AS STRING) AS key", "to_json(struct(*)) AS value").
  writeStream
  .format("kafka")
  .option("topic", "topic-one")
  .option("kafka.bootstrap.servers", "localhost:9092")
  .start()

SyntaxError: invalid syntax (<ipython-input-9-090b7f70e295>, line 1)